In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
df = pd.read_json('datasets/df_limpio.json', lines=True) 
df_aux = pd.read_json('datasets/df_limpio.json', lines=True) 

In [3]:
all_genres = set()
for genres_list in df_aux['genres']:
    if isinstance(genres_list, list):
        all_genres.update(genres_list)

# Crear variables dummy para cada género en el DataFrame
for genre in all_genres:
    df_aux[genre] = df_aux['genres'].apply(lambda x: 1 if genre in x else 0)

In [4]:
df_aux

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,...,Action,Early Access,Design &amp; Illustration,Photo Editing,Adventure,Casual,Strategy,Software Training,Massively Multiplayer,Animation &amp; Modeling
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],...,1,0,0,0,0,1,1,0,0,0
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",...,0,0,0,0,0,0,1,0,0,0
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",...,0,0,0,0,0,1,0,0,0,0
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],...,1,0,0,0,1,1,0,0,0,0
4,None,,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",...,0,0,0,0,0,1,1,0,0,0
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",...,0,0,0,0,0,1,1,0,0,0
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",...,0,0,0,0,0,0,0,0,0,0
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",...,0,0,0,0,0,1,0,0,0,0


In [19]:
df_aux.columns

Index(['discount_price', 'price', 'early_access', 'metascore',
       'anio_lanzamiento', 'Video Production', 'Audio Production', 'Education',
       'Free to Play', 'Utilities', 'Racing', 'Web Publishing', 'Accounting',
       'Sports', 'Simulation', 'RPG', 'Indie', 'Action', 'Early Access',
       'Design &amp; Illustration', 'Photo Editing', 'Adventure', 'Casual',
       'Strategy', 'Software Training', 'Massively Multiplayer',
       'Animation &amp; Modeling'],
      dtype='object')

In [18]:
#df_aux = df_aux.drop('genres', axis=1)
#df_aux = df_aux.drop('title', axis=1)
#df_aux = df_aux.drop('url', axis=1)
#df_aux = df_aux.drop('release_date', axis=1)
#df_aux = df_aux.drop('reviews_url', axis=1)
#df_aux = df_aux.drop('id', axis=1)
df_aux = df_aux.drop('publisher', axis=1)
df_aux = df_aux.drop('app_name', axis=1)
df_aux = df_aux.drop('tags', axis=1)
df_aux = df_aux.drop('specs', axis=1)
df_aux = df_aux.drop('developer', axis=1)
df_aux = df_aux.drop('sentiment', axis=1)


In [7]:
# Filtrar los videojuegos sin precio y convertir el precio a valores numéricos
df_aux['price'] = pd.to_numeric(df_aux['price'], errors='coerce')
df_aux = df_aux.dropna(subset=['price'])

In [8]:
# Convertir el anio_lanzamiento a valores numéricos
df_aux['anio_lanzamiento'] = pd.to_numeric(df_aux['anio_lanzamiento'], errors='coerce')

In [9]:
df_aux = df_aux.dropna(subset=['anio_lanzamiento'])

In [10]:
# Filtrar los videojuegos sin precio y convertir el precio a valores numéricos
df_aux['metascore'] = pd.to_numeric(df_aux['metascore'], errors='coerce')
df_aux = df_aux.dropna(subset=['metascore'])

In [11]:
# Obtener el conjunto de entrenamiento (videojuegos con precio)
X_train = df_aux[['anio_lanzamiento', 'early_access', 'metascore', 'Design &amp; Illustration', 'Accounting',
       'Early Access', 'Racing', 'Strategy', 'Photo Editing', 'Utilities',
       'RPG', 'Action', 'Massively Multiplayer', 'Software Training',
       'Audio Production', 'Free to Play', 'Web Publishing',
       'Animation &amp; Modeling', 'Education', 'Indie', 'Adventure', 'Casual',
       'Simulation', 'Sports', 'Video Production']]
y_train = df_aux['price']

In [12]:
# Entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [13]:
def predecir_precio(anio_lanzamiento, genre, early_access, metascore):
    # Crear un DataFrame con los parámetros proporcionados
    data = pd.DataFrame({
        'anio_lanzamiento': [anio_lanzamiento],
        'early_access': [early_access],
        'metascore': [metascore]
    })

    # Agregar todas las columnas de géneros presentes en X_train y establecerlas en 0
    for genre_column in X_train.columns:
        data[genre_column] = 0

    # Establecer 1 en la columna correspondiente al género proporcionado
    data.loc[0, genre] = 1

    # Realizar la predicción
    predicted_price = model.predict(data)


    return predicted_price[0]

In [14]:
#precio_predicho= calcular_rmse(2016, 'Adventure', False)
#print("Precio predicho:", precio_predicho)
#print("RMSE:", rmse)
#print(predecir_precio(1999,'Indie', False))
precio_predicho= predecir_precio(1999,'Indie', False, 85)
print("Precio predicho:", precio_predicho)
#print("RMSE:", rmse)

Precio predicho: -2420.204998563783


In [20]:
df_aux.corr()

,discount_price,price,early_access,metascore,anio_lanzamiento,Video Production,Audio Production,Education,Free to Play,Utilities,...,Action,Early Access,Design &amp; Illustration,Photo Editing,Adventure,Casual,Strategy,Software Training,Massively Multiplayer,Animation &amp; Modeling
discount_price,1.000000,0.990536,NaN,0.913146,-0.158282,NaN,NaN,NaN,NaN,NaN,...,0.376404,NaN,NaN,NaN,0.614132,-0.376404,0.376404,NaN,NaN,NaN
price,0.990536,1.000000,-0.002960,0.167704,0.346013,NaN,NaN,NaN,0.165423,NaN,...,0.014244,-0.002960,NaN,NaN,-0.021384,-0.154456,0.043013,NaN,0.107726,NaN
early_access,NaN,-0.002960,1.000000,0.019195,-0.025710,NaN,NaN,NaN,0.315389,NaN,...,0.029949,1.000000,NaN,NaN,0.008323,-0.009290,0.015360,NaN,0.156239,NaN
metascore,0.913146,0.167704,0.019195,1.000000,-0.111800,NaN,NaN,NaN,0.037466,NaN,...,-0.039215,0.019195,NaN,NaN,-0.045612,-0.018467,0.017393,NaN,0.047520,NaN
anio_lanzamiento,-0.158282,0.346013,-0.025710,-0.111800,1.000000,NaN,NaN,NaN,0.009932,NaN,...,0.019570,-0.025710,NaN,NaN,0.208263,0.049505,-0.086654,NaN,0.012818,NaN
Video Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Audio Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Education,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Free to Play,NaN,0.165423,0.315389,0.037466,0.009932,NaN,NaN,NaN,1.000000,NaN,...,-0.024981,0.315389,NaN,NaN,-0.015065,-0.014698,0.014041,NaN,0.397955,NaN
Utilities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
